# Practical case - Food Orders

This notebook only contains the function required by that statement with all models. For more information, see [02-food-orders-details](./02-food-orders-details.ipynb) notebook.

### Statement (written in spanish) 

Para este ejercicio se va a imaginar que se trabaja para una empresa de envíos de comida, presente
en todo el territorio nacional, con miles de pedidos cada día. Dicha empresa tiene un fichero histórico
con todas las peticiones de comida que los clientes han realizado mediante el chat de su web en los
últimos meses. Necesitan analizar en tiempo real qué comidas están pidiendo los usuarios y qué
ingredientes tenían, ya que en la cadena de stock de alimentos es necesario realizar una previsión para
no quedarse sin platos cocinados. 

Se ha calculado que el impacto en las ventas cada vez que uno de
los platos deja de estar disponible es del 7% de pérdidas en esa semana, debido al abandono de la web
de pedidos por parte del cliente. Por tanto, es de vital importancia poder realizar automáticamente
estimaciones al respecto.

El objetivo es programar una función que reciba como input un texto de usuario y devuelva los
fragmentos de texto (chunks) que hagan referencia a las comidas y cantidades que ha solicitado. No es
necesario, ni es el objetivo de este ejercicio, construir un clasificador de intención previo a esta
función, sino simplemente una función que presuponemos recibe una frase con la intención
`Pedir_comida`. Tampoco es objetivo normalizar la salida (por ej.: no es necesario convertir 'tres' a '3'
ni 'pizzas' a 'pizza'). Es, por tanto, un ejercicio de mínimos.

    Por ejemplo: “quiero 3 bocadillos de anchoas y 2 pizzas” →
    [
        {comida:'bocadillo', ingrediente:'anchoas', cantidad:3},
        {comida:'pizza', ingrediente:'null', cantidad:2}
    ]
    
Por tanto, la salida de la función será un array con diccionarios de 2 elementos (`comida` y `cantidad`).
Cuando una cantidad no sea detectada, se pondrá su valor a '1' como valor por defecto.

Se deberá comenzar la práctica por el nivel más básico de dificultad (`RegexParser`) y, en caso de
conseguirlo, añadir los siguientes niveles de forma sucesiva. De esta forma, el entregable contendrá
todas y cada una de las tres formas de solucionar el problema. No basta, por tanto, con incluir, por
ejemplo, únicamente un `NaiveBayesClassifier`, hay que incluir también las otras dos formas si se
quiere obtener la máxima puntuación. Se trata simplemente de una práctica y, por tanto, no se espera
como resultado un sistema de alta precisión listo para usar en producción, sino simplemente una
aproximación básica que permita ejecutar las tres formas de resolver el problema.

Este ejercicio hay que hacerlo con textos de entrenamiento en español, pero teniendo en cuenta que
la precisión de los POS taggers en castellano de NLTK es muy mala. Por tanto, el alumno no debe
frustrarse por no obtener buenos resultados, como hemos dicho anteriormente se trata simplemente de
un ejercicio teórico y podemos suponer que, con un mejor analizador, podríamos obtener mejores
resultados.

Para llevar a cabo la práctica, deberá construirse una cadena NLP con NLTK, con los siguientes
elementos:
    - segmentación de frases,
    - tokenización,
    - POS tagger (analizador mofológico para el español).

A continuación, los POS tags obtenidos serán usados por el `RegexParser`, el `UnigramParser`, el
`BigramParser` y el `NaiveBayesClassifier`.

In [1]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

import numpy as np
import nltk
from nltk.corpus import cess_esp


# Define the getFoodOrders() function required by the statement. 
# That practical case ask us what food is deliveried and its amount. 
# That function is able to give it from a chunked sentence. 
# It is important to know that with that function, we assume the following structures in a sentence:

    # 1. [...] Cantidad Comida [...]
    # 2. [...] Comida [...]

# In 2 case, we assume that `Cantidad = 1`. 
def getFoodOrders(chunked_sentence) :
    delivery = []
    dic = {}
    default_cantidad = 1

    chunks = [ chunk for chunk in nltk.chunk.tree2conlltags(chunked_sentence) 
                  if 'Cantidad' in chunk[2] or 'Comida' in chunk[2] ]
    
    i = 0
    while (i < len(chunks)):
        chunk = chunks[i]
        w, t, c = chunk
        
        if c == 'B-Cantidad' :
            dic['cantidad'] = w

        if c == 'B-Comida' :
            if 'cantidad' not in dic :
                dic['cantidad'] = default_cantidad

            dic['comida'] = w
            
            j = i+1
                  
            condition = (j < len(chunks))
            while (condition) :
                w, t, c = chunks[j]
                
                if c == 'I-Comida' :
                    if 'ingredientes' not in dic :
                        dic['ingredientes'] = w
                    else :
                        dic['ingredientes'] = dic['ingredientes'] + " " + w
        
                j += 1
                condition = (j < len(chunks) and c == 'I-Comida')
   
            delivery.append(dic)
            dic = {}

        i += 1
        
    return delivery



# Define createIOBCorpus() function which give us the IOB Corpus from corpus
# using 'pos_tagger' like a POS tagger and 'regex_parser' like a RegexParser 
def createIOBCorpus(corpus, pos_tagger, regex_parser) :

    iob_corpus = []

    # For each sentence in corpus
    for sent in corpus :
        # Tokenize the sentence
        tokens = nltk.word_tokenize(sent)

        # Tag the sentence
        tagged_sent = pos_tagger.tag(tokens)

        # Parse tagged_sent
        chunked_sent = regex_parser.parse(tagged_sent)

        iob_corpus.append(chunked_sent)
    
    return iob_corpus



# Define UnigramChunker class
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data) 

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
    
    
# Define BigramChunker class
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data) 

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
    
    
# Define NaiveBayesChunker class
class ConsecutivePosTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history) 
                train_set.append( (featureset, tag) )
                history.append(tag)
                
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class NaiveBayesChunker(nltk.ChunkParserI): 
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutivePosTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)
    
    
# Define pos_features() function using only the current POS tag of the word.
def pos_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}



# Create initialize() function to initialize all models
def initialize() :
    
    # Load all tagged sentences of Spanish CESS corpus
    sents = cess_esp.tagged_sents()

    # POS Tagger Training
    # Split the Spanish CESS corpus between training and testing dataset
    training = []
    testing = []

    for i in range(len(sents)) :
        if i % 10 :
            training.append(sents[i])
        else :
            testing.append(sents[i])

    # Import HiddenMarkovModelTagger
    from nltk.tag.hmm import HiddenMarkovModelTagger

    # Create the Spanish POS tagger using HMM Tagger
    spanish_pos_tagger = HiddenMarkovModelTagger.train(training)
    
    # Define the grammar for RegexParser
    grammar = r"""
        Comida: {<s?n[cp\.].*><s.*>?<s?[na][cp\.].*>*}
                {<s?n[cp\.].*>}
        Cantidad: {<d[in].*>}
                  {<Z.*>}  
    """

    # Create the RegexParser
    regex_parser = nltk.RegexpParser(grammar)
    
    
    # Create a corpus with sentences for UnigramChunker, BigramChunker and NaiveBayesChunker training.
    corpus = [
        "Me gustaría comer una tortilla de patatas",
        "¿Nos pones 3 tocinos, 4 pechugas?",
        "5 repollos, 12 sardinas y 8 jalapeños",
        "Estamos indecisos, pero puedes ponernos mientras tres raciones de patatas fritas",
        "¿Sería tan amable de servirnos catorce tostadas?",
        "Queremos 2 pollos con caracoles, una ternera, tres patos y 2 pimientos con arroz",
        "Nuestro pedido es: 10 hamburguesas con queso, 20 pizzas y 3 patatas",
        "Yo quiero 9 yogures, 12 ensaladas, un pimiento, 4 chorizos, 12 empanadillas de atún, 8 crespillos y 3 alubias",
        "Me gustaría comer: kiwi con ensalada",
        "Él quiere pedir 3 tostadas, 2 tomates y 6 uvas",
        "Ellos han pedido 2 higos, 1 salmón con caracoles y arroz",
        "Tú has pedido 4 fajitas de pollo, dos emperadores con patatas y tres rúculas",
        "2 macarrones, 3 quesos, una guindilla, cuatro pulpos y quince verduras",
        "Mi abuela quiere emperador con ensalada",
        "Mi tía quiere comer marisco y carne",
        "2 cebollas, una calabaza y ocho tomates",
        "tres pollos, dos corderos, cuatro cerdos y tres macarrones",
        "Pepinillos, calabaza, navajas, sepia y gulas",
        "Me gustaría comer bonito a la plancha con patatas asadas",
        "Pídete cinco pomelos, tres calabacines, 8 mangos, seis melocotones"
        "Langosta, langostinos y almejas"
    ]

    # Create IOB Corpus
    iob_corpus = createIOBCorpus(corpus, spanish_pos_tagger, regex_parser)

    
    # Create UnigramChunker
    unigram_chunker = UnigramChunker(iob_corpus)
    
    # Create BigramChunker
    bigram_chunker = BigramChunker(iob_corpus)
    
    # Create NaiveBayesChunker
    naive_chunker = NaiveBayesChunker(iob_corpus)
    
    return spanish_pos_tagger, regex_parser, unigram_chunker, bigram_chunker, naive_chunker

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
if __name__ == "__main__" :
    pos_tagger, regex_parser, unigram_chunker, bigram_chunker, naive_chunker = initialize()
    
    condition = True
    
    while (condition) :
        sentence = input("Type the food order (in spanish):\n(type 'quit' for exit)\n")
        
        condition = (sentence != "quit")
        
        if condition :
            tokens = nltk.word_tokenize(sentence)
            tagged = pos_tagger.tag(tokens)

            parsed = naive_chunker.parse(tagged)

            regex = regex_parser.parse(tagged)
            unigram = unigram_chunker.parse(tagged)
            bigram = bigram_chunker.parse(tagged)
            naive = naive_chunker.parse(tagged)

            print("The food order is:")
            print("RegexParser:")
            print(getFoodOrders(regex))

            print()
            print("UnigramChunker:")
            print(getFoodOrders(unigram))

            print()
            print("BigramChunker:")
            print(getFoodOrders(bigram))

            print()
            print("NaiveBayesChunker:")
            print(getFoodOrders(naive))
            
            print()
            print("-------------------")
            print()
        else :
            print()
            print("Thank you, good bye!")

Type the food order (in spanish):
(type 'quit' for exit)
Yo quiero tres arroces con pollo, cuatro sopas y 14 pepinos
The food order is:
RegexParser:
[{'cantidad': 'tres', 'comida': 'arroces', 'ingredientes': 'con pollo'}, {'cantidad': 'cuatro', 'comida': 'sopas'}, {'cantidad': '14', 'comida': 'pepinos'}]

UnigramChunker:
[{'cantidad': 'tres', 'comida': 'arroces', 'ingredientes': 'con'}, {'cantidad': 1, 'comida': 'pollo'}, {'cantidad': 'cuatro', 'comida': 'sopas'}, {'cantidad': '14', 'comida': 'pepinos'}]

BigramChunker:
[{'cantidad': 'tres', 'comida': 'arroces', 'ingredientes': 'con pollo'}, {'cantidad': 'cuatro', 'comida': 'sopas'}, {'cantidad': '14', 'comida': 'pepinos'}]

NaiveBayesChunker:
[{'cantidad': 'tres', 'comida': 'arroces', 'ingredientes': 'con'}, {'cantidad': 1, 'comida': 'pollo'}, {'cantidad': 'cuatro', 'comida': 'sopas'}, {'cantidad': '14', 'comida': 'pepinos'}]

-------------------

Type the food order (in spanish):
(type 'quit' for exit)
Ellos van a pedir un guiso de c